# Transcript from Lecture 5, October 7, 2021


In [1]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2021-fall/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d




np.set_printoptions(precision = 4)

# Easy matrices to solve with

In [2]:
# Unit lower triangular (ones on the diagonal)
L = L = np.array([[1,0,0,0], [.5,1,0,0], [0,.5,1,0], [-.5,-.5,0,1]])
L

array([[ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ,  0. ],
       [ 0. ,  0.5,  1. ,  0. ],
       [-0.5, -0.5,  0. ,  1. ]])

In [3]:
# Make a right-hand side 
b = np.random.rand(4)
print("b:", b)

b: [9.2272e-01 7.8387e-05 5.1428e-01 4.8955e-02]


In [4]:
# Solve with unit lower triangular
x = cs111.Lsolve(L, b)
print("x:", x)
print()
print("b - Lx:", b - L@x)  # residual vector

x: [ 0.9227 -0.4613  0.7449  0.2797]

b - Lx: [0. 0. 0. 0.]


In [5]:
# Upper triangular (not unit diagonal)
U = np.array([[2,7,1,8], [0,2,8,1], [0,0,8,2], [0,0,0,8]])
U

array([[2, 7, 1, 8],
       [0, 2, 8, 1],
       [0, 0, 8, 2],
       [0, 0, 0, 8]])

In [6]:
# Make a new right-hand side 
b = np.random.rand(4)
print("b:", b)

b: [0.9057 0.6007 0.3195 0.8634]


In [7]:
# Solve with general upper triangular
x = cs111.Usolve(U, b)
print("x:", x)
print()
print("b - Ux:", b - U@x)

x: [-0.6665  0.1946  0.013   0.1079]

b - Ux: [0. 0. 0. 0.]


# Solving Ax = b when you know a factorization into easy matrices

In [8]:
# Now a different matrix A and right-hand side vector b
A = np.array([[ 2. ,  7. ,  1. ,  8. ],
       [ 1. ,  5.5,  8.5,  5. ],
       [ 0. ,  1. , 12. ,  2.5],
       [-1. , -4.5, -4.5,  3.5]])
A

array([[ 2. ,  7. ,  1. ,  8. ],
       [ 1. ,  5.5,  8.5,  5. ],
       [ 0. ,  1. , 12. ,  2.5],
       [-1. , -4.5, -4.5,  3.5]])

In [9]:
L @ U

array([[ 2. ,  7. ,  1. ,  8. ],
       [ 1. ,  5.5,  8.5,  5. ],
       [ 0. ,  1. , 12. ,  2.5],
       [-1. , -4.5, -4.5,  3.5]])

In [10]:
A - L@U

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

Since **A==L@U**, we can substitute **L@U** for **A**:

> **A @ x == b**  is the same as  **(L @ U) @ x == b**
    
Since matrix and vector multiplication is associative, we can rearrange the parentheses:

> **(L @ U) @ x == b**  is the same as   **L @ (U @ x) == b**
    
Now define **y==U@x**. Substituting, we see that 

> **L @ (U @ x) == b**  is the same as  **L @ y == b**
    
We solve first for **y**, then for **x**.

In [11]:
print("b:", b)
print()

y = cs111.Lsolve(L, b)
print("y:", y)
print()

x = cs111.Usolve(U, y)
print("x:", x)
print()

print("A @ x:", A @ x)
print()

print("b - A @ x:", b - A @ x)

b: [0.9057 0.6007 0.3195 0.8634]

y: [0.9057 0.1479 0.2455 1.3902]

x: [-0.3691  0.0381 -0.0128  0.1738]

A @ x: [0.9057 0.6007 0.3195 0.8634]

b - A @ x: [ 1.1102e-16  0.0000e+00  0.0000e+00 -1.1102e-16]


Where do all those tiny numbers come from? Floating-point! We'll get back to that later.

# Factoring A = LU by Gaussian elimination

In [12]:
# Just to see that our LUfactor does the right thing

LL, UU = cs111.LUfactorNoPiv(A)

In [13]:
LL

array([[ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ,  0. ],
       [ 0. ,  0.5,  1. ,  0. ],
       [-0.5, -0.5,  0. ,  1. ]])

In [14]:
L

array([[ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ,  0. ],
       [ 0. ,  0.5,  1. ,  0. ],
       [-0.5, -0.5,  0. ,  1. ]])

In [15]:
UU - U

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [16]:
# Now let's try a new example. 
n = 6

A = np.random.random((n,n))*100
A

array([[56.0049,  4.3766, 73.0235, 92.4498, 48.5622, 54.8524],
       [78.4349, 30.052 , 29.2583, 99.4134, 70.8832,  9.412 ],
       [ 0.5388, 57.8575, 20.2247, 59.8533, 46.0114, 58.8706],
       [17.9577, 20.1103, 68.6195,  7.0729, 70.2326, 64.9586],
       [98.025 ,  7.376 , 86.7564, 41.4854, 54.63  , 65.162 ],
       [ 2.1294, 95.6386, 99.0477, 27.7796, 64.5426, 89.7163]])

In [17]:
b = np.random.random(n)
print("b:", b)

b: [0.008  0.6689 0.6368 0.3487 0.4378 0.4994]


In [18]:
L, U = cs111.LUfactorNoPiv(A)

In [19]:
L

array([[ 1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 1.4005,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.0096,  2.4168,  1.    ,  0.    ,  0.    ,  0.    ],
       [ 0.3206,  0.782 ,  0.522 ,  1.    ,  0.    ,  0.    ],
       [ 1.7503, -0.0119, -0.2139,  1.3654,  1.    ,  0.    ],
       [ 0.038 ,  3.9909,  1.9781,  1.7134,  1.2159,  1.    ]])

In [20]:
U

array([[ 56.0049,   4.3766,  73.0235,  92.4498,  48.5622,  54.8524],
       [  0.    ,  23.9226, -73.0111, -30.0625,   2.8718, -67.4089],
       [  0.    ,   0.    , 195.9733, 131.6182,  38.6037, 221.2546],
       [  0.    ,   0.    ,   0.    , -67.7671,  32.2646, -15.4121],
       [  0.    ,   0.    ,   0.    ,   0.    , -66.1299,  36.729 ],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    , -99.2587]])

In [21]:
print("b:", b)
print()

y = cs111.Lsolve(L, b)
print("y:", y)
print()

x = cs111.Usolve(U, y)
print("x:", x)
print()

print("A @ x:", A @ x)
print()

print("b - A @ x:", b - A @ x)

b: [0.008  0.6689 0.6368 0.3487 0.4378 0.4994]

y: [ 0.008   0.6578 -0.953   0.3293 -0.2218 -0.5354]

x: [ 0.0071  0.0071 -0.0101 -0.0031  0.0064  0.0054]

A @ x: [0.008  0.6689 0.6368 0.3487 0.4378 0.4994]

b - A @ x: [ 1.1102e-16 -2.2204e-16  0.0000e+00  1.1102e-16  0.0000e+00 -6.6613e-16]


<b>What is the asymptotic running time of cs111.LUfactorNoPiv(A)?

# Permuting rows of A : Partial pivoting

In [22]:
A = np.array([[1,1,2], [1,1,3], [2,3,4]])
A

array([[1, 1, 2],
       [1, 1, 3],
       [2, 3, 4]])

In [23]:
L, U = cs111.LUfactorNoPiv(A)

AssertionError: pivot is zero, can't continue

In [24]:
# LU factorization with partial pivoting ('partial pivoting' means 'permute rows, not columns, of A')
L, U, p = cs111.LUfactor(A)

In [25]:
L

array([[1. , 0. , 0. ],
       [0.5, 1. , 0. ],
       [0.5, 1. , 1. ]])

In [26]:
U

array([[ 2. ,  3. ,  4. ],
       [ 0. , -0.5,  1. ],
       [ 0. ,  0. , -1. ]])

In [28]:
# The permutation of the rows of A
print("p:", p)

p: [2 1 0]


In [29]:
L @ U

array([[2., 3., 4.],
       [1., 1., 3.],
       [1., 1., 2.]])

In [30]:
A

array([[1, 1, 2],
       [1, 1, 3],
       [2, 3, 4]])

In [31]:
A[p, :]

array([[2, 3, 4],
       [1, 1, 3],
       [1, 1, 2]])

In [32]:
n = A.shape[0]
b = np.random.random(n)
print("b:", b)

b: [0.2715 0.5577 0.045 ]


In [33]:
# A complete solve with partial pivoting
print("b:", b)
print()

print("b[p]:", b[p])
print()

y = cs111.Lsolve(L, b[p])
print("y:", y)
print()

x = cs111.Usolve(U, y)
print("x:", x)
print()

print("A @ x:", A @ x)
print()

print("b - A @ x:", b - A @ x)

b: [0.2715 0.5577 0.045 ]

b[p]: [0.045  0.5577 0.2715]

y: [ 0.045   0.5352 -0.2862]

x: [ 0.1969 -0.4979  0.2862]

A @ x: [0.2715 0.5577 0.045 ]

b - A @ x: [ 0.0000e+00 -1.1102e-16 -1.1102e-16]


<b>What is the asymptotic running time of L, U, p = cs111.LUfactor(A)?

# Putting it all together: LUsolve

In [34]:
print("b:", b)
print()

x, rel_res = cs111.LUsolve(A, b)
print("x:", x)
print()

print("A @ x:", A @ x)
print()

print("b - A @ x:", b - A @ x)

b: [0.2715 0.5577 0.045 ]

x: [ 0.1969 -0.4979  0.2862]

A @ x: [0.2715 0.5577 0.045 ]

b - A @ x: [ 0.0000e+00 -1.1102e-16 -1.1102e-16]


<b>What is the asymptotic running time of L, U, p = cs111.LUsolve(A)?

# How good is the answer? Measuring accuracy

In [ ]:
# Next time!